# Random Walks in Higher-Order Networks

In [1]:
import pathpy as pp
from pprint import pprint

We can use `pathpy` to simulate random walks on higher-order networks. Let us first create a simple Higher-Order Network:

In [3]:
n = pp.Network(directed=False)
n.add_edge('a', 'b', weight=1, uid='a-b')
n.add_edge('b', 'c', weight=1, uid='b-c')
n.add_edge('c', 'a', weight=2, uid='c-a')
n.add_edge('c', 'd', weight=1, uid='c-d')
n.add_edge('d', 'a', weight=1, uid='d-a')

n2 = pp.HigherOrderNetwork()
v1 = pp.HigherOrderNode(n.edges['a-b'], uid='a-b')
v2 = pp.HigherOrderNode(n.edges['b-c'], uid='b-c')
v3 = pp.HigherOrderNode(n.edges['c-a'], uid='c-a')
v4 = pp.HigherOrderNode(n.edges['c-d'], uid='c-d')
v5 = pp.HigherOrderNode(n.edges['d-a'], uid='d-a')

n2.add_edge(v1, v2, uid='a-b-c', weight=1)
n2.add_edge(v2, v3, uid='b-c-a', weight=1)
n2.add_edge(v2, v4, uid='b-c-d', weight=0)
n2.add_edge(v3, v1, uid='c-a-b', weight=1)
n2.add_edge(v4, v5, uid='c-d-a', weight=1)
n2.add_edge(v5, v1, uid='d-a-b', weight=1)

To simulate a random in the higher-order network, we can use the same class as for a standard, first-order network:

In [4]:
rw2 = pp.processes.RandomWalk(n2, weight='weight')

We can inspect the transition matrix as follows:

In [6]:
print(rw2.matrix_pd())

     a-b  b-c  c-a  c-d  d-a
a-b  0.0  1.0  0.0  0.0  0.0
b-c  0.0  0.0  1.0  0.0  0.0
c-a  1.0  0.0  0.0  0.0  0.0
c-d  0.0  0.0  0.0  0.0  1.0
d-a  1.0  0.0  0.0  0.0  0.0


We can use the following code to simulate a random walk in the higher-order network. The generated `Path` object is automatically mapped to the first-order nodes. We have to specify a second-order node as the start node (or the number of randomly chosen second-order nodes). 

Based on the higher-order network above, a transition from `b` via `c` to `d` can never occur:

In [9]:
p = rw2.generate_walk(steps=20, start_node=n2.nodes['b-c'])
pprint([ v.uid for v in p.nodes ])

['c',
 'a',
 'b',
 'c',
 'a',
 'b',
 'c',
 'a',
 'b',
 'c',
 'a',
 'b',
 'c',
 'a',
 'b',
 'c',
 'a',
 'b',
 'c',
 'a',
 'b']


Conversely, the transition from `b` via `c` to `d` will occur in the corresponding first-order random walk:

In [15]:
rw = pp.processes.RandomWalk(n, weight='weight')
p = rw.generate_walk(steps=10, start_node=n.nodes['b'])
pprint([ v.uid for v in p.nodes ])

['b', 'b', 'a', 'd', 'a', 'b', 'c', 'd', 'd', 'a', 'a']
